In [5]:
!pip install torch

In [14]:
import transformers as ppb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import torch
import math

In [15]:
import numpy as np
import pandas as pd
#tqdm.pandas(desc='pandas bar')
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models import Word2Vec
import gc
import time

#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
import nltk

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.naive_bayes import MultinomialNB

#lightgbm
import lightgbm as lgb

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras

#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#tqdm.pandas(desc='pandas bar')
import warnings
warnings.filterwarnings('ignore')

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

<Figure size 432x288 with 0 Axes>

In [16]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
df = pd.read_csv("drugsComTrain_raw.tsv", sep="\t", header=None)
test_df = pd.read_csv("drugsComTest_raw.tsv", sep="\t", header=None)

In [18]:
# Truncate long sentences to 128 tokens
X = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=128, truncation=True)))
y = np.array(df[1])
#del df

X_test = test_df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=128, truncation=True)))
y_test = np.array(test_df[1])
#del test_df

ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
# One hot Encoding of y
encoder = LabelEncoder()
encoder.fit(y)

y = encoder.transform(y)
y = to_categorical(y)

# One hot Encoding of y test
y_oh = encoder.transform(y_test)
y_oh = to_categorical(y_oh)

In [ ]:
X.shape

In [13]:
def GetEmbeddings(tokenizedBatch):
    max_len = 0
    for i in tokenizedBatch.values:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenizedBatch.values])
    
    attention_mask = np.where(padded != 0, 1, 0)
    
    input_ids = torch.tensor(padded).to(torch.long)  
    attention_mask = torch.tensor(attention_mask)

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    features = last_hidden_states[0][:,0,:].numpy()
    return features

In [18]:
BATCH_SIZE = 1000
all_embeddings = []
all_embeddings_test = []

# Process Training Set Embeddings
batches = math.ceil(X.shape[0] / BATCH_SIZE)

for i in range(1, batches+1):
    print("Generating Embeddings for Batch:",i,"of", batches)
    batchEmbeddings = GetEmbeddings(X[(i-1)*BATCH_SIZE:i*BATCH_SIZE])
    all_embeddings.append(batchEmbeddings)

# Process Test Set Embeddings
batches = math.ceil(X_test.shape[0] / BATCH_SIZE)

for i in range(1, batches+1):
    print("Generating Test Embeddings for Batch:",i,"of", batches)
    batchEmbeddings = GetEmbeddings(X_test[(i-1)*BATCH_SIZE:i*BATCH_SIZE])
    all_embeddings_test.append(batchEmbeddings)

Generating Embeddings for Batch: 1 of 162
Generating Embeddings for Batch: 2 of 162
Generating Embeddings for Batch: 3 of 162
Generating Embeddings for Batch: 4 of 162
Generating Embeddings for Batch: 5 of 162
Generating Embeddings for Batch: 6 of 162
Generating Embeddings for Batch: 7 of 162
Generating Embeddings for Batch: 8 of 162
Generating Embeddings for Batch: 9 of 162
Generating Embeddings for Batch: 10 of 162
Generating Embeddings for Batch: 11 of 162
Generating Embeddings for Batch: 12 of 162
Generating Embeddings for Batch: 13 of 162
Generating Embeddings for Batch: 14 of 162
Generating Embeddings for Batch: 15 of 162
Generating Embeddings for Batch: 16 of 162
Generating Embeddings for Batch: 17 of 162
Generating Embeddings for Batch: 18 of 162
Generating Embeddings for Batch: 19 of 162
Generating Embeddings for Batch: 20 of 162
Generating Embeddings for Batch: 21 of 162
Generating Embeddings for Batch: 22 of 162
Generating Embeddings for Batch: 23 of 162
Generating Embedding

Generating Test Embeddings for Batch: 27 of 54
Generating Test Embeddings for Batch: 28 of 54
Generating Test Embeddings for Batch: 29 of 54
Generating Test Embeddings for Batch: 30 of 54
Generating Test Embeddings for Batch: 31 of 54
Generating Test Embeddings for Batch: 32 of 54
Generating Test Embeddings for Batch: 33 of 54
Generating Test Embeddings for Batch: 34 of 54
Generating Test Embeddings for Batch: 35 of 54
Generating Test Embeddings for Batch: 36 of 54
Generating Test Embeddings for Batch: 37 of 54
Generating Test Embeddings for Batch: 38 of 54
Generating Test Embeddings for Batch: 39 of 54
Generating Test Embeddings for Batch: 40 of 54
Generating Test Embeddings for Batch: 41 of 54
Generating Test Embeddings for Batch: 42 of 54
Generating Test Embeddings for Batch: 43 of 54
Generating Test Embeddings for Batch: 44 of 54
Generating Test Embeddings for Batch: 45 of 54
Generating Test Embeddings for Batch: 46 of 54
Generating Test Embeddings for Batch: 47 of 54
Generating Te

In [19]:
all_embeddings = np.concatenate(all_embeddings, axis=0)
all_embeddings_test = np.concatenate(all_embeddings_test, axis=0)

In [20]:
np.save('bert_embeddings.npy', all_embeddings)
np.save('y.npy', y)
np.save('bert_embeddings_test.npy', all_embeddings_test)
np.save('y_test.npy', y_oh)

## Train Model

In [1]:
import tensorflow as tf
import tensorflow.keras as keras 
from keras.layers import Input, Lambda, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
import keras.backend as K
#from keras.optimizers import adam, sgd
from tensorflow.keras.optimizers import Adam, SGD

In [6]:
all_embeddings = np.load('bert_embeddings.npy')
y = np.load('y.npy')
all_embeddings_test = np.load('bert_embeddings_test.npy')
y_oh = np.load('y_test.npy')

In [7]:
#sgd = sgd(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
optim = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [8]:
def build_model(): 
    embedding = Input(shape=(768,), dtype="float")
    dense1 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    dense2 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense1)
    dense3 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense2)
    dense4 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense3)
    dense5 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense4)
    dense6 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense5)
    dense7 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense6)
    dense8 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense7)
    dense9 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense8)
    dense10 = Dense(1000, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense9)
    pred = Dense(3, activation='sigmoid')(dense9)
    model = Model(inputs=[embedding], outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'], )
    return model

In [9]:
model_bert = build_model()

In [10]:
model_bert.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense (Dense)               (None, 1000)              769000    
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_4 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_5 (Dense)             (None, 1000)              100100

## Change to training set

In [11]:
es = EarlyStopping(monitor='loss', patience=50)
cb_list = [es]

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model_bert.fit(all_embeddings, y, epochs=10, batch_size=10000, 
                             validation_split = 0.001, callbacks=cb_list)
    model_bert.save_weights('model_bert_weights.h5')

In [12]:
history = model_bert.fit(all_embeddings, y, epochs=10, batch_size=10000, 
                             validation_split = 0.001, callbacks=cb_list)

Epoch 1/10
17/17 [==============================] - 22s 1s/step - loss: 7.7277 - accuracy: 0.5796 - val_loss: 5.4893 - val_accuracy: 0.5679
Epoch 2/10
17/17 [==============================] - 22s 1s/step - loss: 4.3168 - accuracy: 0.6040 - val_loss: 3.2651 - val_accuracy: 0.5679
Epoch 3/10
17/17 [==============================] - 24s 1s/step - loss: 2.7062 - accuracy: 0.6040 - val_loss: 2.2518 - val_accuracy: 0.5679
Epoch 4/10
17/17 [==============================] - 24s 1s/step - loss: 2.0077 - accuracy: 0.6401 - val_loss: 1.7940 - val_accuracy: 0.6728
Epoch 5/10
17/17 [==============================] - 26s 2s/step - loss: 1.6878 - accuracy: 0.6460 - val_loss: 1.5785 - val_accuracy: 0.6667
Epoch 6/10
17/17 [==============================] - 25s 1s/step - loss: 1.4790 - accuracy: 0.6621 - val_loss: 1.3813 - val_accuracy: 0.6790
Epoch 7/10
17/17 [==============================] - 25s 1s/step - loss: 1.3265 - accuracy: 0.6694 - val_loss: 1.3723 - val_accuracy: 0.6049
Epoch 8/10
17/17 [==

In [31]:
import matplotlib.pyplot as plt
%matplotlib inline

#acc = history.history['acc']
loss = history.history['loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'g', label='Training Acc')
plt.title('Training and validation Acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()

plt.show()

NameError: name 'acc' is not defined

Predict on Test Data

In [13]:
bs = 10000
batches = math.ceil(all_embeddings_test.shape[0] / bs)

In [ ]:
all_preds = []
all_probs = []
    
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    model_bert.load_weights('model_bert_weights.h5')

    for i in range(1,batches+1):
        print("Predicting Batch",i)
        new_text_pr = all_embeddings_test[(i-1)*bs:i*bs]
        preds = model_bert.predict(new_text_pr)
        all_probs.append(preds)
        preds = encoder.inverse_transform(np.argmax(preds,axis=1))
        all_preds.append(preds)

In [ ]:
results = np.concatenate(all_preds, axis=0)
results_probs = np.concatenate(all_probs, axis=0)

In [ ]:
print("Accuracy: ",sum(results==y_test)/results.shape[0])